<a href="https://colab.research.google.com/github/sravs-2211/Sravani-S_Task4_ML_Internship/blob/main/subtask1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import time

# Load IMDb dataset
NUM_WORDS = 10000
MAXLEN = 500

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=NUM_WORDS)

# Decode to text for traditional model
word_index = imdb.get_word_index()
index_word = {v + 3: k for k, v in word_index.items()}
index_word[0] = '<PAD>'
index_word[1] = '<START>'
index_word[2] = '<UNK>'
index_word[3] = '<UNUSED>'

def decode_review(text_ids):
    return ' '.join([index_word.get(i, '?') for i in text_ids])

X_train_text = [decode_review(x) for x in X_train]
X_test_text = [decode_review(x) for x in X_test]

# TF-IDF for traditional model
tfidf_vectorizer = TfidfVectorizer(max_features=20000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_text)
X_test_tfidf = tfidf_vectorizer.transform(X_test_text)

# Logistic Regression Model
start_lr = time.time()
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train_tfidf, y_train)
lr_pred = lr_model.predict(X_test_tfidf)
end_lr = time.time()

# Evaluation for Logistic Regression
lr_accuracy = accuracy_score(y_test, lr_pred)
lr_f1 = f1_score(y_test, lr_pred)

# Deep Learning (BiLSTM)
X_train_dl = pad_sequences(X_train, maxlen=MAXLEN)
X_test_dl = pad_sequences(X_test, maxlen=MAXLEN)

model = Sequential()
model.add(Embedding(input_dim=NUM_WORDS, output_dim=128, input_length=MAXLEN))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

start_dl_train = time.time()
model.fit(X_train_dl, y_train, epochs=2, batch_size=128, validation_split=0.2)
end_dl_train = time.time()

# Inference timing
start_dl = time.time()
dl_pred_prob = model.predict(X_test_dl, batch_size=128)
dl_pred = (dl_pred_prob > 0.5).astype("int32")
end_dl = time.time()

# Evaluation for BiLSTM
dl_accuracy = accuracy_score(y_test, dl_pred)
dl_f1 = f1_score(y_test, dl_pred)

# Summary
print("\n📊 Model Comparison Results:")
print(f"Logistic Regression - Accuracy: {lr_accuracy:.4f}, F1-Score: {lr_f1:.4f}, Inference Time: {end_lr - start_lr:.2f}s")
print(f"BiLSTM             - Accuracy: {dl_accuracy:.4f}, F1-Score: {dl_f1:.4f}, Inference Time: {end_dl - start_dl:.2f}s")

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/2


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


157/157 ━━━━━━━━━━━━━━━━━━━━ 357s 2s/step - accuracy: 0.6756 - loss: 0.5714 - val_accuracy: 0.8452 - val_loss: 0.3799
Epoch 2/2
157/157 ━━━━━━━━━━━━━━━━━━━━ 345s 2s/step - accuracy: 0.8832 - loss: 0.2988 - val_accuracy: 0.8684 - val_loss: 0.3171
196/196 ━━━━━━━━━━━━━━━━━━━━ 111s 567ms/step

📊 Model Comparison Results:
Logistic Regression - Accuracy: 0.8839, F1-Score: 0.8845, Inference Time: 1.07s
BiLSTM             - Accuracy: 0.8642, F1-Score: 0.8651, Inference Time: 111.55s
